In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import trim_messages, AIMessage, BaseMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
chat = ChatOpenAI(model="gpt-4o-mini")

## trim_message

Reference: https://python.langchain.com/api_reference/core/messages/langchain_core.messages.utils.trim_messages.html

In [3]:
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Hi my name is Yash"),
    AIMessage("Hi Yash, how can I help?"),
    HumanMessage("I just want to know my name"),
    AIMessage("It's Yash, please let me know if you need anything else."),
]

In [4]:
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=chat,
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # start_on="human" makes sure we produce a valid chat history
    start_on="human",
)

In [5]:
chat.get_num_tokens_from_messages(messages)

64

In [6]:
trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I just want to know my name', additional_kwargs={}, response_metadata={}),
 AIMessage(content="It's Yash, please let me know if you need anything else.", additional_kwargs={}, response_metadata={})]

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import trim_messages
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
chat = ChatOpenAI(model="gpt-4o-mini")
parser = StrOutputParser()

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a savage assistant."),
        MessagesPlaceholder(variable_name="history_messages"),
        ("human", "{input_user_message}"),
    ]
)

In [10]:
chain = prompt | chat | parser

In [11]:
trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=chat,
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # start_on="human" makes sure we produce a valid chat history
    start_on="human",
)

## ChatMessageHistory Integration

Different integrations can be used out of the box for ChatMessageHistory.

Reference: https://python.langchain.com/v0.1/docs/integrations/memory/

In [12]:
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    else:
        store[session_id].messages = trimmer.invoke(store[session_id].messages)
    return store[session_id]

In [13]:
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input_user_message",
    history_messages_key="history_messages",
)

In [14]:
response = chain_with_message_history.invoke(
    {"input_user_message": "hi my name is Yash"},
    {"configurable": {"session_id": "1234"}},
)

print(response)

Nice to meet you, Yash! What’s up?


In [15]:
chat_history_obj = get_session_history(session_id="1234")
chat_history_obj.messages

[HumanMessage(content='hi my name is Yash', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice to meet you, Yash! What’s up?', additional_kwargs={}, response_metadata={})]

In [16]:
response = chain_with_message_history.invoke(
    {"input_user_message": "I have 2 friends"},
    {"configurable": {"session_id": "1234"}},
)

print(response)

Cool! Are they the kind of friends who help you move or the ones who just eat all your snacks?


In [17]:
chat_history_obj = get_session_history(session_id="1234")
chat_history_obj.messages

[HumanMessage(content='I have 2 friends', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cool! Are they the kind of friends who help you move or the ones who just eat all your snacks?', additional_kwargs={}, response_metadata={})]

In [18]:
response = chain_with_message_history.invoke(
    {"input_user_message": "whats my name"},
    {"configurable": {"session_id": "1234"}},
)

print(response)

I don’t know your name, but if you tell me, I’ll use it! Or I can just call you "Friend"—it has a nice ring to it.
